<a href="https://colab.research.google.com/github/bogatovam/cv-hse/blob/main/CV_HW_14_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Подготовка окружения

In [1]:
import cv2
import numpy as np

## Задание №1

Система координат камеры получается из мировой поворотом на 45
градусов вокруг оси z и трансляцией на 10 вдоль оси z. Внутренние параметры
камеры: fx=fy=400, cx=960, cy=540. Написать программу, которая выводит на экран
**матрицу проекции камеры** и **целочисленные координаты пикселя на изображении**,
соответствующего трехмерной точке с координатами (10, -10, 100) в мировой
системе координат.

**Решение.** 
>$P = K[R|T]$

>$(u,v,z) = P*(x,y,z,1)$

In [2]:
K = [[400, 0,     960],
     [0,   400,   540],
     [0,   0,     1]]
     
K = np.array(K)

In [3]:
angle = np.deg2rad(45)

R =  [[np.cos(angle),   -np.sin(angle),   0],
      [np.sin(angle),   np.cos(angle),   0],
      [0,               0,               1]]

In [4]:
T = [[0, 0, 10]]

T = np.array(T).T

In [5]:
R_T = np.concatenate((R, T), axis=1)

In [6]:
P = np.matmul(K,R_T)
print("Camera projection matrix:\n{}".format(R_T))

Camera projection matrix:
[[ 0.70710678 -0.70710678  0.          0.        ]
 [ 0.70710678  0.70710678  0.          0.        ]
 [ 0.          0.          1.         10.        ]]


In [7]:
point = np.array([[10, -10, 100, 1]]).T
u_v_1 = np.matmul(P, point)

In [8]:
image_point = [np.round(u_v_1[0][0]/u_v_1[2][0]), np.round(u_v_1[1][0]/u_v_1[2][0])]

In [9]:
print("Pixel coords:\n{}".format(image_point))

Pixel coords:
[1011.0, 540.0]


## Задание №2

Используя прямое линейное преобразование (direct linear
transformation) и SVD, написать программу, которая принимает на вход множество
пар двумерных точек, и выводит на экран соответствующую этим парам матрицу
гомографии.

**Решение.** Каждый `i`-й набор точек запишем в виде матрицы размера`2×9`, которая имеет вид:
$
P_i = \begin{bmatrix}
−x_i&−y_i&−1&0&0&0&x_ix′_i&y_ix′_i&x′_i\\0&0&0&−x_i&−y_i&−1&x_iy′_i&y_iy′_i&y′_i]
\end{bmatrix}
$


И далее объединяем все $P_i$ в матрицу $P$ и решаем с помощью SVD однородное уравнение вида $PH=0$

In [10]:
angle = np.deg2rad(45)

R =  [[np.cos(angle),   -np.sin(angle),   0],
      [np.sin(angle),   np.cos(angle),   0],
      [0,               0,               1]]

R = np.array(R)

print("True homography matrix:\n{}".format(R))

True homography matrix:
[[ 0.70710678 -0.70710678  0.        ]
 [ 0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]]


In [11]:
def normalize(x):
  return [(x[0][0]/x[0][2], x[0][1]/x[0][2]), (x[1][0]/x[1][2], x[1][1]/x[1][2])]

In [12]:
def findHomography(points_list):
  A = []
  for points in points_list:
    x, y = points[0]
    u, v = points[1]
    A.append([-x, -y, -1, 0, 0, 0, x*u, y*u, u])
    A.append([0, 0, 0, -x, -y, -1, x*v, y*v, v])

  A = np.asarray(A, dtype=np.float)
  d, u_, vt = cv2.SVDecomp(A)

  x_len = (np.abs(d) < 1e-10).sum()
  if x_len == 0:
    print("There is no solution")
  else:
    x_arr = vt[-x_len:]
    for i in range(x_len):
      h_i = np.reshape(x_arr[i]/x_arr[i][-1], (3,3))
      print("Homography matrix #{}:\n{}\n".format(i + 1, h_i))
      test = A.dot(x_arr[i])
      test[test < 1e-10] = 0
      print("The next values must be ZERO: {}\n\n".format(test))

In [13]:
x_vec = np.random.rand(3, 5) * 10

x_vec_ = R.dot(x_vec)
points_list = np.array(list(map(normalize, zip(x_vec.T, x_vec_.T))))

print("Normalized points \n{}\n".format(points_list))

findHomography(points_list)

Normalized points 
[[[ 0.89259107  1.18223752]
  [-0.20481097  1.46712537]]

 [[ 1.5374177   1.22707266]
  [ 0.21944708  1.95478988]]

 [[ 2.07999365  1.44571846]
  [ 0.44850029  2.49305495]]

 [[ 0.90608315  1.00773452]
  [-0.07187837  1.35327346]]

 [[ 2.10210145  0.31565042]
  [ 1.26321163  1.70960874]]]

Homography matrix #1:
[[ 7.07106781e-01 -7.07106781e-01  1.15762981e-16]
 [ 7.07106781e-01  7.07106781e-01  1.90141611e-15]
 [ 1.48745130e-16 -4.11564711e-16  1.00000000e+00]]

The next values must be ZERO: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]




## Задание №3

Два изображения одной и той же сцены сделаны одной и той же
камерой. Второй снимок сделан после поворота камеры на 30 градусов вокруг оси
x относительно начала координат системы отсчета, связанной с камерой (без
сдвига). Найти матрицу гомографии между двумя изображениями.

**Решение.** 
> $S_2 = RS_1 + T$

Но в нашем случае $T = 0$. Тогда матрица гомографии между изображениями это матрица поворота:
> $H = R$


In [14]:
angle = np.deg2rad(30)

R =  [[1,   0,               0            ],
      [0,   np.cos(angle),  -np.sin(angle)],
      [0,   np.sin(angle),   np.cos(angle)]]

R = np.array(R)

print("Homography matrix:\n{}".format(R))

Homography matrix:
[[ 1.         0.         0.       ]
 [ 0.         0.8660254 -0.5      ]
 [ 0.         0.5        0.8660254]]
